# Hospital overcapacity

In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get state hospital utilization reports

In [99]:
before = pd.read_csv('../hospitals/output/hospital_utilization.csv', dtype={'fac_no':str})

In [100]:
before = before[before['icu_beds'] > 0].sort_values('fac_name', ascending=True)

In [101]:
state_hospital_names = before['fac_name'].to_list()

In [102]:
before.head(10)

,fac_no,fac_name,fac_city,county,license_no,fac_zip,icu_beds,tot_lic_beds
488,106580996,ADVENTIST HEALTH AND RIDEOUT,MARYSVILLE,Yuba,230000126,95901,24,221
77,106150788,ADVENTIST HEALTH BAKERSFIELD,BAKERSFIELD,Kern,120000187,93301,34,254
85,106171049,ADVENTIST HEALTH CLEARLAKE,CLEARLAKE,Lake,110000174,95422,4,32
121,106190323,ADVENTIST HEALTH GLENDALE,GLENDALE,Los Angeles,930000059,91206,42,515
84,106164029,ADVENTIST HEALTH HANFORD,HANFORD,Kings,550001439,93230,22,173
209,106234038,ADVENTIST HEALTH HOWARD MEMORIAL,WILLITS,Mendocino,110000013,95490,4,25
381,106390923,ADVENTIST HEALTH LODI MEMORIAL,LODI,San Joaquin,030000056,95240-5179,10,194
482,106560525,ADVENTIST HEALTH SIMI VALLEY,SIMI VALLEY,Ventura,050000216,93065,24,144
474,106554011,ADVENTIST HEALTH SONORA - GREENLEY,SONORA,Tuolumne,030000094,95370,6,140
221,106281078,ADVENTIST HEALTH ST. HELENA,ST. HELENA,Napa,110000073,94574,12,151


### Get latest hospital capacity report from HHS

In [103]:
current = pd.read_csv('output/current_ca.csv')

In [104]:
current = current[current['total_covid_patients'] > 0].sort_values('hospital', ascending=True)

In [105]:
current['hospital'] = current['hospital'].str.upper()

In [106]:
hhs_hospital_names = current['hospital'].to_list()

In [107]:
current.head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share
187,ADVENTIST HEALTH AND RIDEOUT,Short Term,050133,2020-12-25,Yuba,115,15.0,2.0,0.92,75.0,213.0,0.35
71,ADVENTIST HEALTH BAKERSFIELD,Short Term,050455,2020-12-25,Kern,29,17.0,0.0,1.00,128.0,222.0,0.58
289,ADVENTIST HEALTH CLEARLAKE,Critical Access Hospitals,051317,2020-12-25,Lake,33,1.0,0.0,1.00,2.0,18.0,0.11
18,ADVENTIST HEALTH DELANO,Short Term,050608,2020-12-25,Kern,29,9.0,0.0,1.00,26.0,36.0,0.72
69,ADVENTIST HEALTH HANFORD,Short Term,050121,2020-12-25,Kings,31,9.0,2.0,0.91,91.0,155.0,0.59
304,ADVENTIST HEALTH HOWARD MEMORIAL,Critical Access Hospitals,051310,2020-12-25,Mendocino,45,1.0,0.0,1.00,1.0,19.0,0.05
141,ADVENTIST HEALTH LODI MEMORIAL,Short Term,050336,2020-12-25,San Joaquin,77,8.0,2.0,0.87,49.0,108.0,0.45
138,ADVENTIST HEALTH SIMI VALLEY,Short Term,050236,2020-12-25,Ventura,111,14.0,8.0,0.67,59.0,130.0,0.45
265,ADVENTIST HEALTH SONORA,Short Term,050335,2020-12-25,Tuolumne,109,1.0,3.0,0.50,8.0,45.0,0.18
282,ADVENTIST HEALTH ST HELENA,Short Term,050013,2020-12-25,Napa,55,2.0,7.0,0.56,6.0,42.0,0.14


---

In [108]:
len(before)

345

In [109]:
len(current)

314

In [125]:
hhs_merge = pd.merge(current, before, left_on='hospital', right_on='fac_name', how='left')

In [126]:
state_merge = pd.merge(current, before, left_on='hospital', right_on='fac_name', how='right')

In [127]:
len(state_merge)

345

In [128]:
len(hhs_merge)

315

In [129]:
state_merge[['hospital', 'fac_name']].to_csv('output/state_names.csv', index=False)

In [130]:
hhs_merge[['hospital', 'fac_name']].to_csv('output/hhs_names.csv', index=False)

---

In [139]:
hhs_merge[hhs_merge['county_y'] == 'Los Angeles'][['hospital', 'county_x', 'all_patients', 'tot_lic_beds']]

,hospital,county_x,all_patients,tot_lic_beds
15,ALHAMBRA HOSPITAL MEDICAL CENTER,Los Angeles,65.0,144.0
20,ANTELOPE VALLEY HOSPITAL,Los Angeles,230.0,420.0
29,BEVERLY HOSPITAL,Los Angeles,144.0,202.0
33,CASA COLINA HOSPITAL,Los Angeles,81.0,99.0
35,CEDARS-SINAI MEDICAL CENTER,Los Angeles,830.0,889.0
36,CENTINELA HOSPITAL MEDICAL CENTER,Los Angeles,247.0,362.0
43,CITY OF HOPE HELFORD CLINICAL RESEARCH HOSPITAL,Los Angeles,178.0,217.0
45,COAST PLAZA HOSPITAL,Los Angeles,48.0,117.0
47,COLLEGE MEDICAL CENTER,Los Angeles,126.0,221.0
50,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Los Angeles,49.0,81.0
